In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchtext
from torchtext import data
from torchtext.datasets import PennTreebank

In [2]:
TEXT = data.Field(sequential=True, tokenize='spacy', lower=True)
train, valid, test = PennTreebank.splits(TEXT)
TEXT.build_vocab(train)
vocab_size = len(TEXT.vocab)

ngram_size = 4

train_iter, valid_iter, test_iter = data.BPTTIterator.splits((train, valid, test),
                                                            batch_size=512,
                                                            bptt_len=ngram_size,
                                                            repeat=False)

downloading ptb.train.txt


ptb.train.txt: 5.10MB [00:00, 46.9MB/s]
ptb.valid.txt:   0%|          | 0.00/154k [00:00<?, ?B/s]

downloading ptb.valid.txt


ptb.valid.txt: 400kB [00:00, 10.3MB/s]                   
ptb.test.txt: 450kB [00:00, 11.5MB/s]                   


downloading ptb.test.txt


In [0]:
class Bengio2003(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(Bengio2003, self).__init__()
        self.C = nn.Embedding(vocab_size, embedding_dim)
        self.g = nn.Linear(embedding_dim * context_size, vocab_size)
        self.softmax = nn.Softmax()
        
    def forward(self, inputs):
        out = self.C(inputs)
        out = torch.cat(tuple([out[i] for i in range(0, ngram_size)]), 1)
        out = self.g(torch.tanh(out))
        return F.log_softmax(out, dim=1)

In [0]:
loss_function = nn.NLLLoss()
model = Bengio2003(vocab_size, 100, ngram_size)
optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [0]:
for epoch in range(1):
    
    for i, batch in enumerate(train_iter):
        optimizer.zero_grad()
        
        text, target = batch.text, batch.target[-1]
        output = model(text)
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()